# Traffic Sign Classification with Keras

Keras exists to make coding deep neural networks simpler. To demonstrate just how easy it is, you’re going to use Keras to build a convolutional neural network in a few dozen lines of code.

You’ll be connecting the concepts from the previous lessons to the methods that Keras provides.

## Dataset

The network you'll build with Keras is similar to the example that you can find in Keras’s GitHub repository that builds out a [convolutional neural network for MNIST](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py). 

However, instead of using the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, you're going to use the [German Traffic Sign Recognition Benchmark](http://benchmark.ini.rub.de/?section=gtsrb&subsection=news) dataset that you've used previously.

You can download pickle files with sanitized traffic sign data here.

## Overview

Here are the steps you'll take to build the network:

1. First load the data.
2. Build a feedforward neural network to classify traffic signs.
3. Build a convolutional neural network to classify traffic signs.

Keep an eye on the network’s accuracy over time. Once the accuracy reaches the 98% range, you can be confident that you’ve built and trained an effective model.

## Load the Data

Start by importing the data from the pickle file.

In [ ]:
import keras

In [ ]:
# Load pickled data
import pickle
import os
import numpy as np
from sklearn.model_selection import train_test_split

# TODO: fill this in based on where you saved the training and testing data
training_file = './train.p'
testing_file = './test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert(X_train.shape[0] == y_train.shape[0]), "The number of images is not equal to the number of labels."
assert(X_train.shape[1:] == (32,32,3)), "The dimensions of the images are not 32 x 32 x 3."

## Normalize the data

Now that you've loaded the training data, normalize the input so that it has a mean of 0 and a range between -0.5 and 0.5.

In [ ]:
# TODO: Implement data normalization here.
def normalize_images(X):
    """
    Normalize the image data with Min-Max scaling to a range of [-0.5, 0.5]
    :param image_data: The image data to be normalized
    :return: Normalized image data
    """
    a = -0.5;b = 0.5; Xmin = 0.0;Xmax = 255.0;
    Xm=np.mean(X)
    Xp = X - Xm
    Xmin=np.min(Xp)
    Xmax=np.max(Xp)
    Xp = a + (Xp-Xmin)*(b-a)/(Xmax-Xmin)
    return Xp

X_train=normalize_images(X_train)
X_test=normalize_images(X_test)

# one-hot encoding of labels
def to_categorical(y, nb_classes=None):
    '''Convert class vector (integers from 0 to nb_classes) to binary class matrix, for use with categorical_crossentropy.
    # Arguments
        y: class vector to be converted into a matrix
        nb_classes: total number of classes
    # Returns
        A binary matrix representation of the input.
    '''
    y = np.array(y, dtype='int')
    if not nb_classes:
        nb_classes = np.max(y)+1
    Y = np.zeros((len(y), nb_classes))
    for i in range(len(y)):
        Y[i, y[i]] = 1.
    return Y

Y_train = to_categorical(y_train)
Y_test  = to_categorical(y_test)

# split training set in validation and a new training set
X_train, X_val, Y_train, Y_val = train_test_split(
    X_train, Y_train, test_size=0.25, random_state=42)


# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert(round(np.mean(X_train)) == 0), "The mean of the input data is: %f" % np.mean(X_train)
assert(np.min(X_train) == -0.5 and np.max(X_train) == 0.5), "The range of the input data is: %.1f to %.1f" % (np.min(X_train), np.max(X_train))

In [ ]:
# Save the data for easy access
pickle_file = 'preprocessed_data.p'
if not os.path.isfile(pickle_file):
    print('Saving data to pickle file...')
    try:
        with open(pickle_file, 'wb') as pfile:
            pickle.dump(
                {
                    'X_train': X_train,
                    'Y_train': Y_train,
                    'X_val': X_val,
                    'Y_val': Y_val,
                    'X_test': X_test,
                    'Y_test': Y_test
                },
                pfile, pickle.HIGHEST_PROTOCOL)
    except Exception as e:
        print('Unable to save data to', pickle_file, ':', e)
        raise

print('Data cached in pickle file.')

## Build a Two-Layer Feedfoward Network

The code you've written so far is for data processing, not specific to Keras. Here you're going to build Keras-specific code.

Build a two-layer feedforward neural network, with 128 neurons in the fully-connected hidden layer. 

To get started, review the Keras documentation about [models](https://keras.io/models/sequential/) and [layers](https://keras.io/layers/core/).

The Keras example of a [Multi-Layer Perceptron](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py) network is similar to what you need to do here. Use that as a guide, but keep in mind that there are a number of differences.

In [ ]:
import pickle
pickle_file = 'preprocessed_data.p'

with open(pickle_file, 'rb') as f:
    pickle_data = pickle.load(f)
    X_train = pickle_data['X_train']
    Y_train = pickle_data['Y_train']
    X_val = pickle_data['X_val']
    Y_val = pickle_data['Y_val']
    X_test = pickle_data['X_test']
    Y_test = pickle_data['Y_test']
    del pickle_data  # Free up memory

print('Data and modules loaded.')

In [ ]:
# TODO: Build a two-layer feedforward neural network with Keras here.
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import Adam

input_dim=32*32*3
nb_classes=43

model = Sequential()
model.add(Dense(128, input_dim=input_dim, name='hidden1'))
model.add(Activation('relu'))
model.add(Dense(nb_classes, name='output'))
model.add(Activation('softmax'))

model.summary()

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(model.get_layer(name="hidden1").input_shape == (None, 32*32*3)), "The input shape is: %s" % model.get_layer(name="hidden1").input_shape
assert(model.get_layer(name="output").output_shape == (None, 43)), "The output shape is: %s" % model.get_layer(name="output").output_shape 

## Train the Network
Compile and train the network for 2 epochs. [Use the `adam` optimizer, with `categorical_crossentropy` loss.](https://keras.io/models/sequential/)

Hint 1: In order to use categorical cross entropy, you will need to [one-hot encode the labels](https://github.com/fchollet/keras/blob/master/keras/utils/np_utils.py).

Hint 2: In order to pass the input images to the fully-connected hidden layer, you will need to [reshape the input](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py).

Hint 3: Keras's `.fit()` method returns a `History.history` object, which the tests below use. Save that to a variable named `history`.

In [ ]:
X_train = X_train.reshape(-1, input_dim)
print(X_train.shape)
X_val = X_val.reshape(-1, input_dim)
print(X_val.shape)

In [ ]:
batch_size = 128
nb_epoch = 2

adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer=adam,
          loss='categorical_crossentropy',
          metrics=['accuracy'])

history = model.fit(X_train, Y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1, validation_data=(X_train, Y_train))

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['acc'][0] > 0.5), "The training accuracy was: %.3f" % history.history['acc']

## Validate the Network
Split the training data into a training and validation set.

Measure the [validation accuracy](https://keras.io/models/sequential/) of the network after two training epochs.

Hint: [Use the `train_test_split()` method](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) from scikit-learn.

In [ ]:
# TODO: Compile and train the model to measure validation accuracy.

history = model.fit(X_train, Y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1, validation_data=(X_val, Y_val))

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(round(X_train.shape[0] / float(X_val.shape[0])) == 3), "The training set is %.3f times larger than the validation set." % X_train.shape[0] / float(X_val.shape[0])
assert(history.history['val_acc'][0] > 0.6), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

**Validation Accuracy**: 0.87

## Congratulations
You've built a feedforward neural network in Keras!

Don't stop here! Next, you'll add a convolutional layer to drive.py.

## Convolutions
Build a new network, similar to your existing network. Before the hidden layer, add a 3x3 [convolutional layer](https://keras.io/layers/convolutional/#convolution2d) with 32 filters and valid padding.

Then compile and train the network.

Hint 1: The Keras example of a [convolutional neural network](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py) for MNIST would be a good example to review.

Hint 2: Now that the first layer of the network is a convolutional layer, you no longer need to reshape the input images before passing them to the network. You might need to reload your training data to recover the original shape.

Hint 3: Add a [`Flatten()` layer](https://keras.io/layers/core/#flatten) between the convolutional layer and the fully-connected hidden layer.

In [ ]:
import pickle
import numpy as np

pickle_file = 'preprocessed_data.p'

with open(pickle_file, 'rb') as f:
    pickle_data = pickle.load(f)
    X_train = pickle_data['X_train']
    Y_train = pickle_data['Y_train']
    X_val = pickle_data['X_val']
    Y_val = pickle_data['Y_val']
    X_test = pickle_data['X_test']
    Y_test = pickle_data['Y_test']
    del pickle_data  # Free up memory

print('Data and modules loaded.')


# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert(X_train.shape[0] == Y_train.shape[0]), "The number of images is not equal to the number of labels."
assert(X_train.shape[1:] == (32,32,3)), "The dimensions of the images are not 32 x 32 x 3."

In [ ]:
# TODO: Re-construct the network and add a convolutional layer before the first fully-connected layer.
# TODO: Compile and train the model.
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout,Convolution2D,MaxPooling2D,Flatten
from keras.optimizers import Adam

# number of classes
nb_classes = 43
# input image dimensions
img_rows, img_cols, img_ch = 32, 32, 3
# number of convolutional filters to use and theird
nb_filter = 32
# convolution kernel size
kernel_size = (3, 3)

model = Sequential()
model.add(Convolution2D(nb_filter, kernel_size[0], kernel_size[1], 
                        border_mode='valid', 
                        input_shape=(img_rows,img_cols,img_ch),
                        name='conv1'))
model.add(Activation('relu',name='relu1'))
model.add(Flatten())
model.add(Dense(128, name='hidden1'))
model.add(Activation('relu',name='relu2'))
model.add(Dense(nb_classes, name='output'))
model.add(Activation('softmax',name='softmax'))

model.summary()

adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer=adam,
          loss='categorical_crossentropy',
          metrics=['accuracy'])


In [ ]:
# number of epochs to train
nb_epoch=2
#batch size
batch_size = 128

history = model.fit(X_train, Y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1, validation_data=(X_val, Y_val))

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][0] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

**Validation Accuracy**: 0.95

## Pooling
Re-construct your network and add a 2x2 [pooling layer](https://keras.io/layers/pooling/#maxpooling2d) immediately following your convolutional layer.

Then compile and train the network.

In [ ]:
import pickle
import numpy as np

pickle_file = 'preprocessed_data.p'

with open(pickle_file, 'rb') as f:
    pickle_data = pickle.load(f)
    X_train = pickle_data['X_train']
    Y_train = pickle_data['Y_train']
    X_val = pickle_data['X_val']
    Y_val = pickle_data['Y_val']
    X_test = pickle_data['X_test']
    Y_test = pickle_data['Y_test']
    del pickle_data  # Free up memory

print('Data and modules loaded.')

# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert(X_train.shape[0] == Y_train.shape[0]), "The number of images is not equal to the number of labels."
assert(X_train.shape[1:] == (32,32,3)), "The dimensions of the images are not 32 x 32 x 3."

In [ ]:
# TODO: Re-construct the network and add a pooling layer after the convolutional layer.
from keras.models import Sequential
from keras.layers import Dense, Activation,Convolution2D,MaxPooling2D,Flatten
from keras.optimizers import Adam

# number of classes
nb_classes = 43
# input image dimensions
img_rows, img_cols, img_ch = 32, 32, 3
# number of convolutional filters to use and theird
nb_filter = 32
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3, 3)


model = Sequential()
model.add(Convolution2D(nb_filter, kernel_size[0], kernel_size[1],border_mode='valid', 
                        input_shape=(img_rows,img_cols,img_ch),name='conv1'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Activation('relu',name='relu1'))
model.add(Flatten())
model.add(Dense(128, input_dim=input_dim, name='hidden1'))
model.add(Activation('relu',name='relu3'))
model.add(Dense(nb_classes, name='output'))
model.add(Activation('softmax',name='softmax'))

model.summary()


adam = Adam(lr=1e-04, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer=adam,
          loss='categorical_crossentropy',
          metrics=['accuracy'])

In [ ]:
nb_epoch=10
batch_size=128

history = model.fit(X_train, Y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1, validation_data=(X_val, Y_val))

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][0] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

In [ ]:
model.save('models/TrafficSigns_1conv_1pool.ckpt')

**Validation Accuracy**: 0.94

## Dropout
Re-construct your network and add [dropout](https://keras.io/layers/core/#dropout) after the pooling layer. Set the dropout rate to 50%.

In [1]:
import pickle
import numpy as np

pickle_file = 'preprocessed_data.p'

with open(pickle_file, 'rb') as f:
    pickle_data = pickle.load(f)
    X_train = pickle_data['X_train']
    Y_train = pickle_data['Y_train']
    X_val = pickle_data['X_val']
    Y_val = pickle_data['Y_val']
    X_test = pickle_data['X_test']
    Y_test = pickle_data['Y_test']
    del pickle_data  # Free up memory

print('Data and modules loaded.')

# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert(X_train.shape[0] == Y_train.shape[0]), "The number of images is not equal to the number of labels."
assert(X_train.shape[1:] == (32,32,3)), "The dimensions of the images are not 32 x 32 x 3."

Data and modules loaded.


In [2]:
# TODO: Re-construct the netMaxPooling2Dwork and add dropout after the pooling layer.
# TODO: Compile and train the model.
import os
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout,Convolution2D,MaxPooling2D,Flatten
from keras.optimizers import Adam


# number of classes
nb_classes = 43
# input image dimensions
img_rows, img_cols, img_ch = 32, 32, 3
# number of convolutional filters to use and theird
nb_filter = 32
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3, 3)


model = Sequential()
model.add(Convolution2D(nb_filter, kernel_size[0], kernel_size[1],border_mode='valid', 
                        input_shape=(img_rows,img_cols,img_ch),name='conv1'))
model.add(Activation('relu',name='relu1'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(128, name='hidden1'))
model.add(Activation('relu',name='relu2'))
model.add(Dense(nb_classes, name='output'))
model.add(Activation('softmax',name='softmax'))

model.summary()

Using TensorFlow backend.


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
conv1 (Convolution2D)            (None, 30, 30, 32)    896         convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
relu1 (Activation)               (None, 30, 30, 32)    0           conv1[0][0]                      
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 15, 15, 32)    0           relu1[0][0]                      
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 15, 15, 32)    0           maxpooling2d_1[0][0]             
___________________________________________________________________________________________

In [4]:
from keras.models import load_model
nb_epoch = 2
batch_size=128

model_name='models/TrafficSigns_1conv_1pool_1dropout.ckpt'

if os.path.isfile(model_name):
    try:
        model = load_model(model_name)
        print('loading trained model ...')
    except Exception as e:
        print('Unable to load model', model_name, ':', e)
        raise    


adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer=adam,
          loss='categorical_crossentropy',
          metrics=['accuracy'])

history = model.fit(X_train, Y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1, validation_data=(X_val, Y_val))

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][-1] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][-1]

loading trained model ...
Train on 29406 samples, validate on 9803 samples
Epoch 1/2
29406/29406 [==============================] - 2s - loss: 0.1888 - acc: 0.9501 - val_loss: 0.1616 - val_acc: 0.9648
Epoch 2/2
29406/29406 [==============================] - 1s - loss: 0.1479 - acc: 0.9598 - val_loss: 0.1340 - val_acc: 0.9682


In [5]:
model.save('models/TrafficSigns_1conv_1pool_1dropout.ckpt')

**Validation Accuracy**: 0.95

## Optimization
Congratulations! You've built a neural network with convolutions, pooling, dropout, and fully-connected layers, all in just a few lines of code.

Have fun with the model and see how well you can do! Add more layers, or regularization, or different padding, or batches, or more training epochs.

What is the best validation accuracy you can achieve?

In [1]:
import pickle
import numpy as np

pickle_file = 'preprocessed_data.p'

with open(pickle_file, 'rb') as f:
    pickle_data = pickle.load(f)
    X_train = pickle_data['X_train']
    Y_train = pickle_data['Y_train']
    X_val = pickle_data['X_val']
    Y_val = pickle_data['Y_val']
    X_test = pickle_data['X_test']
    Y_test = pickle_data['Y_test']
    del pickle_data  # Free up memory

print('Data and modules loaded.')

# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert(X_train.shape[0] == Y_train.shape[0]), "The number of images is not equal to the number of labels."
assert(X_train.shape[1:] == (32,32,3)), "The dimensions of the images are not 32 x 32 x 3."

Data and modules loaded.


In [2]:
import os
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout,Convolution2D,MaxPooling2D,Flatten
from keras.optimizers import Adam


# number of classes
nb_classes = 43
# input image dimensions
img_rows, img_cols, img_ch = 32, 32, 3
# number of convolutional filters to use
nb_filter1 = 32
nb_filter2 = 64
nb_filter3 = 128
# size of pooling area for max pooling
pool_size = (2, 2)
pool_strides = (1,1)
# convolution kernel size
kernel_size = (5, 5)
# number of hidden units in the first fully connected layer
nb_fc1=128
nb_fc2=128


model = Sequential()
model.add(Convolution2D(nb_filter1, kernel_size[0], kernel_size[1],border_mode='same', subsample=(1,1),
                        input_shape=(img_rows,img_cols,img_ch),name='conv1'))
model.add(Activation('relu',name='relu1'))
model.add(MaxPooling2D(pool_size=pool_size,strides=pool_strides,name='maxpool1'))
model.add(Convolution2D(nb_filter2, kernel_size[0], kernel_size[1],border_mode='same',subsample=(1,1),
                        name='conv2'))
model.add(Activation('relu',name='relu2'))
model.add(MaxPooling2D(pool_size=pool_size,strides=None,name='maxpool2'))
model.add(Convolution2D(nb_filter2, kernel_size[0], kernel_size[1],border_mode='same',subsample=(1,1),
                        name='conv3'))
model.add(Activation('relu',name='relu3'))
model.add(MaxPooling2D(pool_size=pool_size,strides=None,name='maxpool3'))
model.add(Flatten(name='flatten'))
model.add(Dropout(0.5,name='dropout1'))
model.add(Dense(nb_fc1, name='hidden1'))
model.add(Activation('relu',name='relu4'))
model.add(Dropout(0.5,name='dropout2'))
model.add(Dense(nb_fc2,  name='hidden2'))
model.add(Activation('relu',name='relu5'))
model.add(Dense(nb_classes, name='output'))
model.add(Activation('softmax',name='softmax'))

model.summary()

Using TensorFlow backend.


loading trained model ...
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
conv1 (Convolution2D)            (None, 32, 32, 32)    2432        convolution2d_input_2[0][0]      
____________________________________________________________________________________________________
relu1 (Activation)               (None, 32, 32, 32)    0           conv1[0][0]                      
____________________________________________________________________________________________________
maxpool1 (MaxPooling2D)          (None, 31, 31, 32)    0           relu1[0][0]                      
____________________________________________________________________________________________________
conv2 (Convolution2D)            (None, 31, 31, 64)    51264       maxpool1[0][0]                   
_________________________________________________________________

In [9]:
nb_epoch = 1000
batch_size=128

model_name='models/TrafficSigns_3conv_3pool_2dropout.ckpt'

if os.path.isfile(model_name):
    try:
        model = load_model(model_name)
        print('loading trained model ...')
    except Exception as e:
        print('Unable to load model', model_name, ':', e)
        raise    


adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer=adam,
          loss='categorical_crossentropy',
          metrics=['accuracy'])

history = model.fit(X_train, Y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=2, validation_data=(X_val, Y_val))

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][-1] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][-1]

loading trained model ...
Train on 29406 samples, validate on 9803 samples
Epoch 1/1000
6s - loss: 0.0213 - acc: 0.9932 - val_loss: 0.0158 - val_acc: 0.9969
Epoch 2/1000
5s - loss: 0.0199 - acc: 0.9943 - val_loss: 0.0153 - val_acc: 0.9971
Epoch 3/1000
5s - loss: 0.0218 - acc: 0.9928 - val_loss: 0.0169 - val_acc: 0.9969
Epoch 4/1000
5s - loss: 0.0193 - acc: 0.9939 - val_loss: 0.0166 - val_acc: 0.9967
Epoch 5/1000
5s - loss: 0.0189 - acc: 0.9935 - val_loss: 0.0165 - val_acc: 0.9970
Epoch 6/1000
5s - loss: 0.0169 - acc: 0.9944 - val_loss: 0.0171 - val_acc: 0.9968
Epoch 7/1000
5s - loss: 0.0174 - acc: 0.9948 - val_loss: 0.0157 - val_acc: 0.9969
Epoch 8/1000
5s - loss: 0.0187 - acc: 0.9943 - val_loss: 0.0164 - val_acc: 0.9970
Epoch 9/1000
5s - loss: 0.0166 - acc: 0.9944 - val_loss: 0.0157 - val_acc: 0.9971
Epoch 10/1000
5s - loss: 0.0191 - acc: 0.9940 - val_loss: 0.0148 - val_acc: 0.9972
Epoch 11/1000
5s - loss: 0.0186 - acc: 0.9938 - val_loss: 0.0164 - val_acc: 0.9970
Epoch 12/1000
5s - lo

In [10]:
model.save(model_name)

**Best Validation Accuracy:** (0.9948)

## Testing
Once you've picked out your best model, it's time to test it.

Load up the test data and use the [`evaluate()` method](https://keras.io/models/model/#evaluate) to see how well it does.

Hint 1: After you load your test data, don't forget to normalize the input and one-hot encode the output, so it matches the training data.

Hint 2: The `evaluate()` method should return an array of numbers. Use the `metrics_names()` method to get the labels.

In [11]:
model.evaluate(X_test, Y_test)

12630/12630 [==============================] - 1s     


[0.13127789783443108, 0.97988915285796097]

**Test Accuracy:** 0.9799

This is a pretty good result. With 97.98 % test accuracy we would have come in place 15 of the original 2011 German Traffic Sign Recognition Benchmark Competition, see <a href="http://benchmark.ini.rub.de/?section=gtsrb&subsection=news">GTSRB</a> and . We have not even used data augmentation, any more sophisticated preprocessing of the images or hyperparameter tuning so far and doing so should further improve the results. In order to do this sytematically we would have to check precisely _what_ traffic signs get classified  incorrectly

But for now we'll leave it at that. A great reference of a high scoring submission to the competition (98.97%) is the 2 convolutional layer network that includes also more advanced and interesting features such as layer skipping on <a href="http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf"> Yann LeCun's website</a> 

## Summary
Keras is a great tool to use if you want to quickly build a neural network and evaluate performance.